## Neural NLP and accounting/finance

In [4]:
import torch

In [5]:
torch.cuda.is_available()

True

In [6]:
import transformers, os, re

In [7]:
from pathlib import Path

In [8]:
from tokenizers import ByteLevelBPETokenizer

In [9]:
#source_dir = "D:/Data/Reasonable_10K/"

In [10]:
dest_dir = "D:/Data/Reasonable_10K/reduced/"

In [11]:
#files = os.listdir(source_dir)

In [12]:
#pattern = r'[0-9]'

In [13]:
"""for file in files:
    document = open(os.path.join(source_dir, file)).read().split('</Header>')[1]
    red_document = re.sub(pattern, '', document)
    open(os.path.join(dest_dir,file),mode='w').write(red_document)"""

"for file in files:\n    document = open(os.path.join(source_dir, file)).read().split('</Header>')[1]\n    red_document = re.sub(pattern, '', document)\n    open(os.path.join(dest_dir,file),mode='w').write(red_document)"

In [14]:
paths = [str(x) for x in Path(dest_dir).glob("**/*.txt")]

In [15]:
# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

In [16]:
# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [17]:
tokenizer.save_model(".","AccBert_model")

['.\\AccBert_model-vocab.json', '.\\AccBert_model-merges.txt']

In [19]:
from tokenizers.processors import BertProcessing

In [20]:
tokenizer = ByteLevelBPETokenizer(
    "AccBert_model-vocab.json",
    "AccBert_model-merges.txt",
)


In [21]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)

In [22]:
tokenizer.enable_truncation(max_length=512)

In [27]:
print(tokenizer.encode("The annual profit was"))

Encoding(num_tokens=6, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [28]:
tokenizer.encode("The annual profit was").tokens

['<s>', 'The', 'Ġannual', 'Ġprofit', 'Ġwas', '</s>']

In [30]:
from torch.utils.data import Dataset

In [35]:
class AccBertDataset(Dataset):
    def __init__(self, evaluate: bool = False):
        tokenizer = ByteLevelBPETokenizer(
            "AccBert_model-vocab.json",
            "AccBert_model-merges.txt",
        )
        tokenizer._tokenizer.post_processor = BertProcessing(
            ("</s>", tokenizer.token_to_id("</s>")),
            ("<s>", tokenizer.token_to_id("<s>")),
        )
        tokenizer.enable_truncation(max_length=512)

        self.examples = []

        src_files = Path(dest_dir).glob("**/*.txt") if evaluate else Path(dest_dir).glob("**/*.txt")
        for src_file in src_files:
            lines = src_file.read_text(encoding="utf-8").splitlines()
            self.examples += [x.ids for x in tokenizer.encode_batch(lines)]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        # We’ll pad at the batch level.
        return torch.tensor(self.examples[i])

In [36]:
test = AccBertDataset()

In [ ]:
test.examples